In [1]:
import camelot
import json
import os
import re

In [2]:
def get_continued_tables(tables, threshold):

    continued_tables = {}
    previous_table = False
    group_counter = 0

    # typical height of a pdf is 842 points and bottom margins are anywhere between 56 and 85 points
    # therefore, accounting for margins, 792
    page_height = 792

    # iterate over the tables
    for i, table in enumerate(tables):

        # if a previous table exists (remember, we start with this as false)
        # and the previous table was on the previous page
        # and the number of columns of both tables is the same
        if previous_table and table.page == previous_table.page + 1 and len(table.cols) == len(previous_table.cols):

            # get the bottom coordinate of the previous table
            # note that for pdfs the origin (0, 0) typically starts from the bottom-left corner of the page,
            # with the y-coordinate increasing as you move upwards
            # this is why for {x0, y0, x1, y1} we need the y0 as the bottom
            previous_table_bottom = previous_table._bbox[1]

            # get the top coordinate of the current table
            # for {x0, y0, x1, y1} we need the y1 as the top
            current_table_top = table._bbox[3]

            # if the previous table ends in the last 15% of the page and the current table starts in the first 15% of the page
            if previous_table_bottom < (threshold / 100) * page_height and current_table_top > (1 - threshold / 100) * page_height:

                # if we don't have started this group of tables
                if (continued_tables.get(group_counter) is None):

                    # start by adding the first table
                    continued_tables[group_counter] = [previous_table]

                # add any of the sunsequent tables to the group
                continued_tables[group_counter].append(table)

            # if this is not a continuation of the previous table
            else:

                # increment the group number
                group_counter += 1;

        # if this is not a continuation of the previous table
        else:

            # increment the group number
            group_counter += 1;

        # the current table becomes the previous table for the next iteration
        previous_table = table

    # transform the dictionary into an array of arrays
    continued_tables = [value for value in continued_tables.values()]

    # return the combined tables
    return continued_tables

In [3]:
def table_to_json(table_data, table_info):
    """Convert table data to JSON format"""
    if not table_data:
        return {}
    
    # Create JSON structure
    json_data = {
        "metadata": {
            "source_file": table_info["source_file"],
            "page": table_info["page"],
            "table_order": table_info["order"],
            "total_rows": len(table_data),
            "total_columns": len(table_data[0]) if table_data else 0
        },
        "headers": [],
        "data": []
    }
    
    # Add headers (first row)
    if len(table_data) > 0:
        headers = [str(cell).strip() for cell in table_data[0]]
        
        # Replace first 3 headers with fixed names
        if len(headers) >= 1:
            headers[0] = "STT"
        if len(headers) >= 2:
            headers[1] = "hang_hoa"
        if len(headers) >= 3:
            headers[2] = "yeu_cau_ky_thuat"
            
        json_data["headers"] = headers
        
        # Add data rows (skip header)
        for i, row in enumerate(table_data[1:], 1):
            row_dict = {}
            for j, cell in enumerate(row):
                # Use header as key, fallback to column index if header is empty
                key = json_data["headers"][j] if j < len(json_data["headers"]) and json_data["headers"][j] else f"column_{j}"
                row_dict[key] = str(cell).strip()
            
            json_data["data"].append({
                "row_index": i,
                "values": row_dict
            })
    
    return json_data

In [4]:
def get_biggest_table(pdf_path, threshold):
    tables = camelot.read_pdf(pdf_path, flavor = 'lattice', pages = 'all')
    continued_tables = get_continued_tables(tables, threshold)

    # get the name of the PDF file we are processing (without the extension)
    pdf_file_name = os.path.splitext(os.path.basename(pdf_path))[0]

    processed = []
    all_table_jsons = []

    # iterate over found tables
    for i, table in enumerate(tables):

        # if table was already processed as part of a group
        if table in processed: continue

        # check if the current table is a continued table
        is_continued = any(table in sublist for sublist in continued_tables)

        # collect all table data (current table + continued tables if any)
        all_table_data = list(table.data)

        # if the current table is a continued table, append all subsequent continued tables data
        if is_continued:

            # get the index of the group in "continued_tables" associated with the current table
            group_index = next(index for index, sublist in enumerate(continued_tables) if table in sublist)

            # iterate over the tables in said group and append their data
            for continued_table in continued_tables[group_index]:

                # skip the current table as it's already added
                if continued_table == table or continued_table in processed: continue

                # append the data of the continued table (skip header for subsequent tables)
                all_table_data.extend(continued_table.data[1:] if len(continued_table.data) > 1 else [])

                # keep track of processed tables
                processed.append(continued_table)

        # convert to JSON
        table_info = {
            "source_file": pdf_file_name,
            "page": table.parsing_report['page'],
            "order": table.parsing_report['order']
        }
        
        json_data = table_to_json(all_table_data, table_info)
        all_table_jsons.append(json_data)
        
        # mark current table as processed
        processed.append(table)

    # find the table with the most rows
    if all_table_jsons:
        largest_table = max(all_table_jsons, key=lambda x: x.get('metadata', {}).get('total_rows', 0))
        
        # return the JSON of the largest table
        print(json.dumps(largest_table, ensure_ascii=False, indent=2))
        return largest_table
    else:
        print("No tables found in the PDF.")
        return None

In [5]:
hello = get_biggest_table("D:/study/LammaIndex/documents/test.pdf",50)

{
  "metadata": {
    "source_file": "test",
    "page": 1,
    "table_order": 1,
    "total_rows": 15,
    "total_columns": 3
  },
  "headers": [
    "STT",
    "hang_hoa",
    "yeu_cau_ky_thuat"
  ],
  "data": [
    {
      "row_index": 1,
      "values": {
        "STT": "I",
        "hang_hoa": "Bộ chuyển đổi nguồn 220VAC/ \n48VDC (kèm theo 02 dàn acquy \n200Ah)",
        "yeu_cau_ky_thuat": ""
      }
    },
    {
      "row_index": 2,
      "values": {
        "STT": "1",
        "hang_hoa": "Yêu cầu chung",
        "yeu_cau_ky_thuat": "-  Các loại thiết bị, vật tư, phụ kiện phải có nguồn gốc xuất xứ rõ ràng, có chứng nhận chất lượng sản \nphẩm của nhà sản xuất. \n-  Thiết bị mới 100% chưa qua sử dụng \n-  Thiết bị phải được sản xuất từ năm 2021 trở lại đây \n-  Tuân thủ tiêu chuẩn IEC 60950-1 \n-  Thuộc loại thiết bị nguồn sử dụng kỹ thuật chuyển mạch, thiết kế dạng Module theo chức năng của \ntừng khối. \n-  Tất  cả  các  khối  đặt  trong  tủ  liên  hoàn,  đồng  bộ của  hãng,  

In [6]:
data = hello["data"]

In [7]:
import uuid
def clean_text(text):
    """Làm sạch text, loại bỏ ký tự xuống dòng thừa"""
    return re.sub(r'\n+', '', text.strip())

def split_requirements(text):
    """Tách các yêu cầu dựa trên dấu gạch đầu dòng"""
    requirements = []
    lines = text.split('\n')
    for line in lines:
        line = line.strip()
        if line.startswith('- '):
            requirements.append(line[2:].strip())
        elif line and not any(line.startswith(prefix) for prefix in ['- ']):
            if requirements:
                requirements[-1] += ' ' + line
            else:
                requirements.append(line)
    return requirements

def generate_random_key():
    """Tạo key random 5 ký tự từ UUID"""
    return str(uuid.uuid4()).replace('-', '')[:5].upper()

def convert_to_new_format(data):
    result = []
    current_product = None
    current_category = None
    
    for item in data:
        values = item['values']
        stt_raw  = values['STT']
        hang_hoa = clean_text(values['hang_hoa'])
        yeu_cau = values['yeu_cau_ky_thuat']


        stt = stt_raw.strip()

        roman_pattern = r'^(VII|VIII|IX|X|XI|XII|I{1,3}|IV|V|VI)\s+(.+)'
        roman_match = re.match(roman_pattern, stt)
        # Nếu STT là số La Mã (I, II, III...) thì đây là tên sản phẩm
        hang_hoa_roman_match = re.match(roman_pattern, hang_hoa)
        if roman_match and not hang_hoa and not yeu_cau:
            if current_product:
                result.append(current_product)
            
            roman_num = roman_match.group(1)  # Số La Mã
            product_name = roman_match.group(2)  # Tên sản phẩm
            
            current_product = {
                "ten_san_pham": product_name,
                "cac_muc": []
            }
            current_category = None
        elif hang_hoa_roman_match and not stt_raw and not yeu_cau:
            if current_product:
                result.append(current_product)
            
            roman_num = hang_hoa_roman_match.group(1)  # Số La Mã
            product_name = hang_hoa_roman_match.group(2)  # Tên sản phẩm
            
            current_product = {
                "ten_san_pham": product_name,
                "cac_muc": []
            }
            current_category = None        
        
        elif stt in ['I', 'II', 'III', 'IV', 'V', 'VI', 'VII', 'VIII', 'IX', 'X', 'XI', 'XII', 'XIII', 'XIV']:
            if current_product:
                result.append(current_product)
            
            current_product = {
                "ten_san_pham": hang_hoa,
                "cac_muc": []
            }
            current_category = None
            
        # Nếu STT là số (1, 2, 3...) thì đây là danh mục
        elif stt.isdigit():
            current_category = {
                "ten_hang_hoa": hang_hoa,
                "thong_so_ky_thuat": {}
            }
            
            # Xử lý yêu cầu kỹ thuật cho danh mục
            if yeu_cau.strip():
                requirements = split_requirements(yeu_cau)
                for req in requirements:
                    key = generate_random_key()  # Tạo key random
                    current_category["thong_so_ky_thuat"][key] = clean_text(req)
            if current_product:
                current_product["cac_muc"].append(current_category)
                
        # Nếu STT trống thì đây là thông số kỹ thuật chi tiết
        elif stt == '' and current_category and hang_hoa:
            # Tạo key random cho thông số kỹ thuật
            key = generate_random_key()
            
            # Làm sạch tên hàng hóa và yêu cầu kỹ thuật
            clean_hang_hoa = clean_text(hang_hoa)
            clean_yeu_cau = clean_text(yeu_cau)
            
            current_category["thong_so_ky_thuat"][key] = [clean_hang_hoa, clean_yeu_cau]
        elif stt == '' and current_category and not hang_hoa:
            if yeu_cau.strip():
                requirements = split_requirements(yeu_cau)
                
                # Lấy key cuối cùng trong thong_so_ky_thuat (nếu có)
                existing_keys = list(current_category["thong_so_ky_thuat"].keys())
                last_key = existing_keys[-1] if existing_keys else None
                
                for req in requirements:
                    clean_req = clean_text(req)
                    
                    # Kiểm tra chữ cái đầu có viết hoa HOẶC có gạch đầu dòng không
                    has_dash = req.strip().startswith('- ')
                    has_uppercase = clean_req and clean_req[0].isupper()
                    
                    if has_uppercase or has_dash:
                        # Chữ đầu viết hoa HOẶC có gạch đầu dòng -> tạo key mới
                        key = generate_random_key()
                        current_category["thong_so_ky_thuat"][key] = clean_req
                        last_key = key
                    else:
                        # Chữ đầu không viết hoa VÀ không có gạch đầu dòng -> nối vào key trước đó
                        if last_key and last_key in current_category["thong_so_ky_thuat"]:
                            current_category["thong_so_ky_thuat"][last_key] += " " + clean_req
                        else:
                            # Nếu không có key trước đó thì vẫn tạo key mới
                            key = generate_random_key()
                            current_category["thong_so_ky_thuat"][key] = clean_req
                            last_key = key
    
    # Thêm sản phẩm cuối cùng
    if current_product:
        result.append(current_product)
    
    return result

# Chuyển đổi dữ liệu
converted_data = convert_to_new_format(data)

In [8]:
converted_data

[{'ten_san_pham': 'Bộ chuyển đổi nguồn 220VAC/ 48VDC (kèm theo 02 dàn acquy 200Ah)',
  'cac_muc': [{'ten_hang_hoa': 'Yêu cầu chung',
    'thong_so_ky_thuat': {'00CF4': 'Các loại thiết bị, vật tư, phụ kiện phải có nguồn gốc xuất xứ rõ ràng, có chứng nhận chất lượng sản phẩm của nhà sản xuất.',
     '4791C': 'Thiết bị mới 100% chưa qua sử dụng',
     '69384': 'Thiết bị phải được sản xuất từ năm 2021 trở lại đây',
     '3D9A2': 'Tuân thủ tiêu chuẩn IEC 60950-1',
     '8D47E': 'Thuộc loại thiết bị nguồn sử dụng kỹ thuật chuyển mạch, thiết kế dạng Module theo chức năng của từng khối.',
     'EB99A': 'Tất  cả  các  khối  đặt  trong  tủ  liên  hoàn,  đồng  bộ của  hãng,  tạo  thành  thiết  bị  cấp  nguồn  và  nạp acquyhoàn chỉnh.',
     '7F060': 'Thời gian bảo hành: theo tiêu chuẩn của nhà sản xuất, tối thiểu 12 tháng.'}},
   {'ten_hang_hoa': 'Cấu hình thiết bị nguồn',
    'thong_so_ky_thuat': {'79196': 'Số lượng khe cắm module chỉnh lưu (Rectifier): ≥ 4',
     'D872C': 'Công suất mỗi module 

In [9]:
context_queries = {}  # Dict chứa thông tin chi tiết theo key
product_key = {}  # Dict lồng: ten_san_pham -> ten_hang_hoa -> list[key]

for item in converted_data:
    ten_san_pham = item['ten_san_pham']
    for muc in item['cac_muc']:
        ten_hang_hoa = muc['ten_hang_hoa']
        thong_so_ky_thuat = muc['thong_so_ky_thuat']
        for key, value in thong_so_ky_thuat.items():
            if isinstance(value, list):
                q = value[0]
                k = value[1]
                value_str = ' '.join(value)
            else:
                q = None
                k = value
                value_str = value

            # Ghi vào context_queries
            context_queries[key] = {
                "ten_san_pham": ten_san_pham,
                "ten_hang_hoa": ten_hang_hoa,
                "value": value_str,
                "yeu_cau_ky_thuat_chi_tiet": k,
                "yeu_cau_ky_thuat": q
            }

            # Ghi vào product_key
            if ten_san_pham not in product_key:
                product_key[ten_san_pham] = {}
            if ten_hang_hoa not in product_key[ten_san_pham]:
                product_key[ten_san_pham][ten_hang_hoa] = []
            product_key[ten_san_pham][ten_hang_hoa].append(key)



In [10]:
context_queries

{'00CF4': {'ten_san_pham': 'Bộ chuyển đổi nguồn 220VAC/ 48VDC (kèm theo 02 dàn acquy 200Ah)',
  'ten_hang_hoa': 'Yêu cầu chung',
  'value': 'Các loại thiết bị, vật tư, phụ kiện phải có nguồn gốc xuất xứ rõ ràng, có chứng nhận chất lượng sản phẩm của nhà sản xuất.',
  'yeu_cau_ky_thuat_chi_tiet': 'Các loại thiết bị, vật tư, phụ kiện phải có nguồn gốc xuất xứ rõ ràng, có chứng nhận chất lượng sản phẩm của nhà sản xuất.',
  'yeu_cau_ky_thuat': None},
 '4791C': {'ten_san_pham': 'Bộ chuyển đổi nguồn 220VAC/ 48VDC (kèm theo 02 dàn acquy 200Ah)',
  'ten_hang_hoa': 'Yêu cầu chung',
  'value': 'Thiết bị mới 100% chưa qua sử dụng',
  'yeu_cau_ky_thuat_chi_tiet': 'Thiết bị mới 100% chưa qua sử dụng',
  'yeu_cau_ky_thuat': None},
 '69384': {'ten_san_pham': 'Bộ chuyển đổi nguồn 220VAC/ 48VDC (kèm theo 02 dàn acquy 200Ah)',
  'ten_hang_hoa': 'Yêu cầu chung',
  'value': 'Thiết bị phải được sản xuất từ năm 2021 trở lại đây',
  'yeu_cau_ky_thuat_chi_tiet': 'Thiết bị phải được sản xuất từ năm 2021 trở l

In [ ]:
product_key

In [ ]:
# Lưu với tên file mặc định
with open('context_prompts.json', 'w', encoding='utf-8') as f:
    json.dump(context_prompts, f, ensure_ascii=False, indent=2)

print(f"✅ Đã lưu {len(context_prompts)} context prompts vào file: context_prompts.json")

In [29]:
from openai import OpenAI
from dotenv import load_dotenv
import re
import os
load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [ ]:
test = dict(list(context_queries.items())[1:20])
test

In [ ]:
test, product_key

In [ ]:
import os
from dotenv import load_dotenv
from llama_index.core import VectorStoreIndex
from llama_index.core.vector_stores import VectorStoreInfo
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import QdrantClient, AsyncQdrantClient
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core.retrievers import VectorIndexAutoRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.settings import Settings
from llama_index.core.vector_stores import (
    MetadataFilter,
    MetadataFilters,
    FilterOperator,
    FilterCondition
)
import time


# Cấu hình LLM và Embedding
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")
Settings.llm = OpenAI(model="gpt-4o-mini")
# Cấu hình client Qdrant
client = QdrantClient(
    url="https://a8bcf78f-0147-411f-aa58-079f863fcd6d.us-west-1-0.aws.cloud.qdrant.io:6333",
    api_key=os.getenv("QDRANT_API_KEY"),
)
aclient = AsyncQdrantClient(
    url="https://a8bcf78f-0147-411f-aa58-079f863fcd6d.us-west-1-0.aws.cloud.qdrant.io:6333",
    api_key=os.getenv("QDRANT_API_KEY"),
)
# Khởi tạo Vector Store
vector_store = QdrantVectorStore(
    collection_name="hello_my_friend",
    client=client,
    aclient=aclient,
)

def retrieve_document(product_line, query_str):
    product_ids = []
    index = VectorStoreIndex.from_vector_store(vector_store=vector_store)
    
    filters_document = MetadataFilters(
        filters=[
            MetadataFilter(key="product_line", operator=FilterOperator.EQ, value=product_line),
            MetadataFilter(key="type", operator=FilterOperator.EQ, value="summary_document"),
        ],
    condition=FilterCondition.AND,
    )
    retriever_document = index.as_retriever(similarity_top_k=5, sparse_top_k=10, verbose=True, enable_hybrid=True, filters=filters_document)
    
    results = retriever_document.retrieve(query_str)
    # print("product: ", results)
    # print("results: ", results)
    for result in results:
        metadata = result.metadata
        product_ids.append(metadata["product_id"])

    return product_ids

def retrieve_chunk(product_ids, query_str):
    index = VectorStoreIndex.from_vector_store(vector_store=vector_store)
    
    filters_chunk = MetadataFilters(
        filters=[
            MetadataFilter(key="product_id", operator=FilterOperator.IN, value=product_ids),
            MetadataFilter(key="type", operator=FilterOperator.EQ, value="chunk_document"),
        ],
        condition=FilterCondition.AND,
    )

    retriever_chunk = index.as_retriever(similarity_top_k=5, verbose=True, filters=filters_chunk)
    
    results = retriever_chunk.retrieve(query_str)
    print("chunk : ", results)
    content = ""
    for i, result in enumerate(results, start=1):
        metadata = result.metadata
        file_name = metadata["file_name"]+ ".pdf"
        page = metadata["page"]
        table = metadata["table_name"]
        figure_name = metadata.get("figure_name")
        text = result.text.strip()
        content += f"Chunk {i} trong file {file_name} tại trang {page}, có chứa bảng {table} và hình {figure_name} có nội dung:\n{text}\n\n"

    return content

def retrieve_product_line(product_name, assistant_id="asst_j5wHMN84dpSLXD2GMH5QifS0"):
    from openai import OpenAI 
    clientOpenAi = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    thread = clientOpenAi.beta.threads.create()
    thread_id = thread.id
    # 2. Gửi message vào thread
    clientOpenAi.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=product_name
    )
    run = clientOpenAi.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=assistant_id,
        tool_choice="auto"  # hoặc thay bằng tool cụ thể nếu cần
        # tool_choice={"type": "function", "function": {"name": "danh_gia_ky_thuat"}}
    )
    run_id = run.id
    # 4. Đợi assistant xử lý xong
    while True:
        run_status = clientOpenAi.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
        if run_status.status == "completed":
            break
        elif run_status.status in ["failed", "cancelled", "expired"]:
            raise Exception(f"Run failed with status: {run_status.status}")
        time.sleep(1)

    # 5. Lấy kết quả trả về từ Assistant
    messages = clientOpenAi.beta.threads.messages.list(thread_id=thread_id)
    for message in reversed(messages.data):  # đảo ngược để lấy kết quả mới nhất trước
        if message.role == "assistant":
            for content in message.content:
                if content.type == "text":
                    # print("product_line: ", content.text.value)
                    return content.text.value

    return None


        




D:\study\LammaIndex\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [32]:
def create_prompt_extract_module(query_str):
    prompt = f"""
You are an expert in hardware product documentation analysis.  
Read the provided text (which can be either a detailed product brochure or a general product requirement) and extract ONLY the core physical hardware components/modules of the system.
 
For each component:
- If the text explicitly includes a model number, code, or exact specification tied to the component → output "<Full Component Name>: <Exact Model(s)/Code(s)>".
- If the text does NOT provide a model number or code → output only "<Full Component Name>".
 
Input:
<<<
{query_str}
>>>
 
Output format:
- <Component Name>[: <Model(s)/Code(s) if available>]
 
Rules:
1. Only include core hardware modules essential for the product’s operation (e.g., Rectifier Module, Controller, AC Input, AC Distribution, DC Distribution, Battery Distribution, Lightning Protection, Cooling System, Battery Bank).
2. Preserve the exact wording of component/module names from the text (do not paraphrase or generalize).
3. Include model numbers, codes, or exact designations only if explicitly stated in the text.  
   - If multiple models exist, list them separated by " / ".
4. If a component has sub-parts (e.g., BLVD/LLVD, Input/Output), keep them as separate lines with their full names.
5. Ignore optional accessories, warranty info, standards compliance, and marketing text unless they are part of the official component name/specification.
7. Do not infer or guess component names—extract only what is explicitly stated.    
"""
    return prompt

In [33]:
from openai import OpenAI
from dotenv import load_dotenv
import re
import os
load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


In [35]:
for product in product_key:
    product_line = retrieve_product_line(product)
    query_str = ""
    items = product_key[product]
    for key in items:
        for item in items[key]:
            if item not in test:
                continue
            query_str += test[item]['value']
    prompt = create_prompt_extract_module(query_str)
    response = client.responses.create(
        model="gpt-4o-mini",
        input=prompt,
        temperature=0
    )
    output_text = response.output_text.strip()
    product_ids = retrieve_document(product_line, query_str)
    print(product_ids)
    print(output_text)
        

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_14304\3376383554.py:92: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = clientOpenAi.beta.threads.create()
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_14304\3376383554.py:95: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  clientOpenAi.beta.threads.messages.create(
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_14304\3376383554.py:100: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = clientOpenAi.beta.threads.runs.create(
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_14304\3376383554.py:109: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run_status = clientOpenAi.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_14304\3376383554.py:117: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  messages = 

['a34a2f4f-7368-11f0-a85b-38f3abb08dd1', '74730619-7368-11f0-9fa1-38f3abb08dd1', 'ef875c94-7371-11f0-a926-38f3abb08dd1', '5e7f9ad1-736a-11f0-b74b-38f3abb08dd1', '4938c279-7371-11f0-9a31-38f3abb08dd1']
- Module chỉnh lưu: ≥ 3 module
- Attomat DC: 
  - Loại 32A: ≥ 02 cái
  - Loại 16A: ≥ 01 cái
  - Loại 63A: ≥ 02 cái
  - Loại 32A: ≥ 03 cái
  - Loại 16A: ≥ 02 cái
  - Attomat cho acquy: 
    - Loại 125A: ≥ 02 cái
- Dải điện áp đầu vào: từ 85VAC ÷ 300VAC
- Dải tần số AC đầu vào: từ 45 Hz ÷ 65 Hz
- Điện áp đầu ra danh định: -48VDC
- Dải điện áp đầu ra: từ -43.2VDC tới -57.6VDC
- Độ ổn định điện áp đầu ra: ≤1%
- Nhiễu băng rộng: ≤20mV rms (5Hz÷1MHz)
- Độ gợn sóng đầu ra (đỉnh – đỉnh): ≤ 250 mV (0÷20MHz)


In [41]:
with open("D:/study/LammaIndex/output/NetSure -731 A41 Brochure.md", "r", encoding="utf-8") as f:
    md_content = f.read()

In [ ]:
md_content

In [43]:
prompt = create_prompt_extract_module(md_content)

In [44]:
response = client.responses.create(
        model="gpt-4o-mini",
        input=prompt,
        temperature=0
)

In [45]:
output_text = response.output_text.strip()
print(output_text)

- Rectifier: R48-3000A3 / R48-3000e3 / R48-3500e3 / R48-3500E4
- Controller: M221S / M830B
- AC Distribution: 
  - Input: 1 x 125 A / 2P
  - Output: 1 x 16 A / 16P
- Battery Distribution: 2 x 125 A / 1P
- DC Distribution: 
  - BLVD: 2 x 63 A / 1P, 4 x 32 A / 1P, 2 x 16 A / 1P
  - LLVD: 2 x 32 A / 1P, 2 x 16 A / 1P
- Lightning Protection: Class C lightning protection (AC side) / Class D lightning protection (DC side)


In [ ]:
for key in items:
    for item in items[key]:
        if item not in test:
            continue
        query = test[item]["value"]
        content = retrieve_chunk(product_ids, query)
        test[item]["relevant_context"] = content

In [38]:
for key in test:
    print(key)

93C6C
8E2C6
17415
D70B1
ABE44
B4706
A09AB
9F080
5F95F
FC80B
5552C
3513E
0C362
80D4F
6D7D8
3323F
97B78
070AF
1F101


In [ ]:
test

In [41]:
SYSTEM_PROMPT = """
Bạn được cung cấp:
- Một hoặc nhiều đoạn văn bản (chunk) từ tài liệu kỹ thuật, kèm metadata: tên file, mục, bảng/hình (nếu có), số trang
- Một yêu cầu kỹ thuật cụ thể.
- Một đoạn văn mẫu.

#Yêu cầu trả lời bằng tiếng việt:
# 1. Tìm thông tin kỹ thuật liên quan trực tiếp đến yêu cầu kỹ thuật.
# 2. Trích xuất giá trị thông số để xác định khả năng đáp ứng theo yêu cầu và trả về đoạn văn tương tự giống đoạn văn mẫu không thêm bớt nhưng thông số phải chính xác trong tài liệu.
# 3. Dẫn chứng rõ: file, section, table/figure name (nếu có), page, nội dung trích dẫn của những tài liệu liên quan, những tài liệu khác không liên quan thì bỏ qua.

# #Output: JSON gồm các trường:
- yeu_cau_ky_thuat
- kha_nang_dap_ung
- tai_lieu_tham_chieu" 

# Ví dụ:
Input:
Yêu cầu: "Số lượng khe cắm module chỉnh lưu (Rectifier): ≥ 4"  
Chunk: "...NetSure 731 A41-S8: 4 rectifier slots (standard), expandable to 6..."  
Metadata:  
- file: "Netsure-731-A41-user-manual.pdf"  
- section: "Table 1-1 Configuration of power system"  
- page: 2"
Đoạn văn mẫu: Số lượng khe cắm module chỉnh lưu (Rectifier): ≥ 4 (ví dụ tìm trong tài liệu số lượng là 5 thì trả về "Số lượng khe cắm module chỉnh lưu (Rectifier): 5")
"""

# Định nghĩa function schema
FUNCTION_SCHEMA = {
    "name": "danh_gia_ky_thuat",
    "description": "Đánh giá khả năng đáp ứng của sản phẩm theo yêu cầu kỹ thuật từ chunk tài liệu.",
    "parameters": {
        "type": "object",
        "properties": {
            "yeu_cau_ky_thuat": {"type": "string"},
            "kha_nang_dap_ung": {"type": "string"},
            "tai_lieu_tham_chieu": {
                "type": "object",
                "properties": {
                    "file": {"type": "string"},
                    "section": {"type": "string"},
                    "table_or_figure": {"type": "string"},
                    "page": {"type": "integer"},
                    "evidence": {"type": "string"}
                },
                "required": ["file", "section", "page", "evidence"]
            }
        },
        "required": ["yeu_cau_ky_thuat", "kha_nang_dap_ung", "tai_lieu_tham_chieu"]
    }
}

In [42]:
# Tạo assistant
# === CREATE ASSISTANT ===
def create_assistant():
    assistant = client.beta.assistants.create(
        name="Technical Document Evaluator",
        instructions=SYSTEM_PROMPT,
        model="gpt-4o-mini",
        tools=[{"type": "function", "function": FUNCTION_SCHEMA}]
    )
    return assistant.id

# === CREATE THREAD ===
def create_thread():
    thread = client.beta.threads.create()
    return thread.id

# === UPDATE ASSISTANT ===
def update_assistant(assistant_id):
    assistant = client.beta.assistants.update(
        assistant_id=assistant_id,
        instructions=SYSTEM_PROMPT,
        model="gpt-4o-mini",
        tools=[{"type": "function", "function": FUNCTION_SCHEMA}]
    )
    return assistant.id

# === EVALUATE TECHNICAL REQUIREMENT ===
def evaluate_technical_requirement(user_prompt, assistant_id):
    # 1. Tạo thread riêng cho mỗi lần gọi
    thread = client.beta.threads.create()
    thread_id = thread.id

    # 2. Gửi message vào thread
    client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=user_prompt
    )

    # 3. Tạo run
    run = client.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=assistant_id,
        tool_choice={"type": "function", "function": {"name": "danh_gia_ky_thuat"}}
    )

    # 4. Chờ assistant xử lý (tối đa 20s)
    for _ in range(20):
        run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run.id)
        if run.status not in ["queued", "in_progress"]:
            break
        time.sleep(1)

    # 5. Lấy arguments trực tiếp
    if run.status == "requires_action":
        call = run.required_action.submit_tool_outputs.tool_calls[0]
        print(f"👉 Assistant đã gọi tool: {call.function.name}")
        print("🧠 Dữ liệu JSON assistant muốn trả về:")
        print(call.function.arguments)
        return call.function.arguments

    elif run.status == "completed":
        messages = client.beta.threads.messages.list(thread_id=thread_id)
        for msg in messages.data:
            print(f"[{msg.role}] {msg.content[0].text.value}")
        return None

    else:
        print(f"Run status: {run.status}")
        return None



In [ ]:
test

In [ ]:
import openai
from openai import OpenAI
import json
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [ ]:
# Ví dụ sử dụng
import time
assistant_id = create_assistant()
print(f"Assistant ID: {assistant_id}")


# Ví dụ user prompt

for key in test:
    if not test[key].get('relevant_context'):
        continue
    data = test[key]
    value = data['value']
    content = data['relevant_context']
    form = data['yeu_cau_ky_thuat_chi_tiet']
    # test[key].pop("content", None)
    
    user_prompt = f'''
    Chunk và metadata: {content}
    Yêu cầu: {value}
    Đoạn văn mẫu: {form}
    '''
    
    # Gọi hàm đánh giá với thread riêng
    result = evaluate_technical_requirement(user_prompt, assistant_id)
    test[key]['response'] = result




In [ ]:
test

In [ ]:
import json

for key in test:
    test[key].pop("content", None)
    response = test[key].get('response')  # Sử dụng get để tránh lỗi nếu không có response
    print(response)
    if response:
        if isinstance(response, str):
            first_json_str = response[:response.find('}}') + 2]
            response = json.loads(first_json_str)
        test[key]["kha_nang_dap_ung"] = response.get('kha_nang_dap_ung', '')
        tai_lieu_tham_chieu = {
            "file": response['tai_lieu_tham_chieu'].get('file', ''),
            "section": response['tai_lieu_tham_chieu'].get('section', ''),
            "table_or_figure": response['tai_lieu_tham_chieu'].get('table_or_figure', ''),
            "page": response['tai_lieu_tham_chieu'].get('page', 0),
            "evidence": response['tai_lieu_tham_chieu'].get('evidence', '')
        }
        test[key]["tai_lieu_tham_chieu"] = tai_lieu_tham_chieu
    test[key].pop("response", None)


In [51]:
def prompt_adapt_or_not(dap_ung_ky_thuat: str) -> str:
    prompt = f"""
Bạn sẽ được cung cấp một danh sách các cặp “yêu cầu kỹ thuật || khả năng đáp ứng” trong file dap_ung_ky_thuat.
Nhiệm vụ của bạn:
1. Với từng cặp, đánh giá xem khả năng đáp ứng có thực sự đáp ứng yêu cầu kỹ thuật không.
2. Tổng hợp kết quả:
  – Nếu tất cả các yêu cầu đều được đáp ứng, trả về "Đáp ứng"
  – Nếu không có yêu cầu nào được đáp ứng, trả về "Không đáp ứng"
  – Nếu chỉ một phần yêu cầu được đáp ứng, trả về theo định dạng "Đáp ứng: x/y = z%", trong đó:
       - x là số yêu cầu được đáp ứng
       - y là tổng số yêu cầu
       - z% là phần trăm làm tròn đến số nguyên gần nhất
📤 Kết quả chỉ trả về dưới dạng JSON với cấu trúc sau: {{
 "đáp ứng kỹ thuật": "<kết quả đánh giá>"
}}

Danh sách các cặp “yêu cầu kỹ thuật || khả năng đáp ứng” : {dap_ung_ky_thuat}
"""
    return prompt

In [52]:
from openai import OpenAI
from dotenv import load_dotenv
import re
import os
load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [53]:
def parse_output_text(output_text: str) -> dict:
    # B1: Loại bỏ phần ```json ... ```
    cleaned = re.sub(r"^```json\n|```$", "", output_text.strip())
    print(cleaned)
    # B2: Giải mã các ký tự escape như \n, \"
    unescaped = cleaned.encode("utf-8")

    # B3: Chuyển thành dict
    return json.loads(unescaped)

In [54]:
results = []
for product in product_key:
    items = product_key[product]
    for key in items:
        dap_ung_ky_thuat = ""
        tai_lieu_tham_chieu = ""
        for item in items[key]:
            if item not in test:
                continue
            yeu_cau_ky_thuat = test[item].get('yeu_cau_ky_thuat_chi_tiet', "")
            kha_nang_dap_ung = test[item].get('kha_nang_dap_ung', '')
            dap_ung_ky_thuat += f"{yeu_cau_ky_thuat} || {kha_nang_dap_ung}\n"
            
            tai_lieu = test[item].get('tai_lieu_tham_chieu', {})
            file = tai_lieu.get("file", "")
            page = tai_lieu.get("page", "")
            table_or_figure = tai_lieu.get("table_or_figure", "")
            evidence = tai_lieu.get("evidence", "")
            
            # Tạo câu mô tả tài liệu
            tai_lieu_text = f"{file}, trang: {page}"
            if table_or_figure:
                tai_lieu_text += f", trong bảng(figure): {table_or_figure}"
            tai_lieu_text += f", evidence: {evidence}\n\n"
            tai_lieu_tham_chieu += tai_lieu_text  # 🔧 thiếu dòng này

        if dap_ung_ky_thuat and tai_lieu_tham_chieu:  # Có thể dùng trực tiếp như bool
            prompt = prompt_adapt_or_not(dap_ung_ky_thuat)
            response = client.responses.create(
                model="gpt-4o-mini",
                input=prompt,
                temperature=0
            )
            output_text = response.output_text.strip()
            output_text = parse_output_text(output_text)
            print(output_text)
            product_key[product][key].append(output_text['đáp ứng kỹ thuật'])
            product_key[product][key].append(tai_lieu_tham_chieu)


{
  "đáp ứng kỹ thuật": "Đáp ứng: 4/6 = 67%"
}

{'đáp ứng kỹ thuật': 'Đáp ứng: 4/6 = 67%'}
{
  "đáp ứng kỹ thuật": "Đáp ứng: 5/5 = 100%"
}

{'đáp ứng kỹ thuật': 'Đáp ứng: 5/5 = 100%'}
{
  "đáp ứng kỹ thuật": "Đáp ứng"
}

{'đáp ứng kỹ thuật': 'Đáp ứng'}
{
  "đáp ứng kỹ thuật": "Đáp ứng: 5/5 = 100%"
}

{'đáp ứng kỹ thuật': 'Đáp ứng: 5/5 = 100%'}


In [55]:
product_key 

{'Bộ chuyển đổi nguồn 220VAC/ 48VDC (kèm theo 02 dàn acquy 200Ah)': {'Yêu cầu chung': ['BCCA8',
   '93C6C',
   '8E2C6',
   '17415',
   'D70B1',
   'ABE44',
   'B4706',
   'Đáp ứng: 4/6 = 67%',
   'Netsure 731 A41 Usermanual.pdf, trang: 11, evidence: The equipment should be unpacked and inspected after it arrives at the installation site. The inspection shall be done by representatives of both the user and Vertiv Co., Ltd. To inspect the equipment, you should open the packing case, take out the packing list and check against the packing list that the equipment is correct and complete. Make sure that the equipment is delivered intact.\n\nnetsure-2100-brochure.pdf, trang: 2, evidence: Specifications are subject to change without notice. November 2018\n\nNetsure-801-brochure.pdf, trang: 4, evidence: | Electrical    | IEC 60950-1, EN 60950-1, UL 60950-1            |\n\nNetsure 731 A41 Usermanual.pdf, trang: 9, trong bảng(figure): None, evidence: The power system is composed of power distrib

In [ ]:
test

In [57]:
product_key_test = {
    'Bộ chuyển đổi nguồn 220VAC/ 48VDC (kèm theo 02 dàn acquy 200Ah)': {'Yêu cầu chung': ['BCCA8',
   '93C6C',
   '8E2C6',
   '17415',
   'D70B1',
   'ABE44',
   'B4706',
   'Đáp ứng: 4/6 = 67%',
   'Netsure 731 A41 Usermanual.pdf, trang: 11, evidence: The equipment should be unpacked and inspected after it arrives at the installation site. The inspection shall be done by representatives of both the user and Vertiv Co., Ltd. To inspect the equipment, you should open the packing case, take out the packing list and check against the packing list that the equipment is correct and complete. Make sure that the equipment is delivered intact.\n\nnetsure-2100-brochure.pdf, trang: 2, evidence: Specifications are subject to change without notice. November 2018\n\nNetsure-801-brochure.pdf, trang: 4, evidence: | Electrical    | IEC 60950-1, EN 60950-1, UL 60950-1            |\n\nNetsure 731 A41 Usermanual.pdf, trang: 9, trong bảng(figure): None, evidence: The power system is composed of power distribution、rectifier modules and controller module.\n\nNetsure 731 A41 Usermanual.pdf, trang: 35, trong bảng(figure): Figure 5 Accessory showing four components of the battery rack, evidence: Để lắp đặt hoàn chỉnh cho bộ cấp nguồn và nạp acquy, các phụ kiện sau đây được sử dụng: Accessory 1 (main frame), Accessory 2 (support bar), Accessory 3 (long bar), Accessory 4 (small bracket), Expansion bolt, và Fastener.\n\nNetsure-731-A41-user-manual.pdf, trang: 1, evidence: NO_CONTENT_HERE\n\n'],
  'Cấu hình thiết bị nguồn': ['A09AB',
   '9F080',
   '5F95F',
   'FC80B',
   'Đáp ứng: 5/5 = 100%',
   'NetSure -731 A41 Brochure.pdf, trang: 1, trong bảng(figure): None, evidence: | Rectifier            | 4 x R48-3000A3 / R48-3000e3 / R48-3500e3 / R48-3500OE4 |\n\nNetsure 731 A41 Usermanual.pdf, trang: 30, evidence: Default: dưới 45℃, 3000W 출력.\n\nnetsure-801-brochure.pdf, trang: 5, trong bảng(figure): Bảng, evidence: Số lượng module chỉnh lưu: 1-20\n\nNetsure 731 A41 Usermanual.pdf, trang: 10, evidence: | Battery MCB                  | 2×125A/1P                       | 2×125A/1P                       | 4×125A/1P                                       |\n\n'],
  'Đầu vào AC': ['5552C',
   '3513E',
   '0C362',
   '80D4F',
   'Đáp ứng',
   'Netsure 731 A41 Usermanual.pdf, trang: 30, evidence: AC input system L＋N＋PE/220Vac；3P＋N＋PE/220Vac\n\nNetsure-731-A41-user-manual.pdf, trang: 30, trong bảng(figure): None, evidence: | AC input | Input voltage range | 85Vac～300Vac |\n\nNetsure 731 A41 Usermanual.pdf, trang: 30, trong bảng(figure): Technical and Engineering Data, evidence: Dải tần số AC đầu vào từ 45Hz đến 65Hz.\n\nnetsure-212-c23-datasheet2.pdf, trang: 1, evidence: Fully protected from external lightning surges\n\n'],
  'Đầu ra DC': ['6D7D8',
   '3323F',
   '97B78',
   '070AF',
   '1F101',
   'Đáp ứng: 5/5 = 100%',
   'Netsure 731 A41 Usermanual.pdf, trang: 30, trong bảng(figure): Technical and Engineering Data, evidence: | Nominal output voltage   | －48Vdc |\n\nnetsure-701-series_datasheet_en-asia.pdf, trang: 1, trong bảng(figure): Output Data, evidence: | Operating Voltage Range  | -42 Vdc to -58Vdc |\n\nNetsure-731-A41-user-manual.pdf, trang: 30, trong bảng(figure): None, evidence: | Parameter category | Parameter | Description |\n| ------------------------------ | ------------------------------------------------ | ------------------------------------------------------------------------------------------------------------------------------- |\n| DC output | Voltage set-point accuracy | ≤1％ |\n\nNetsure 731 A41 Usermanual.pdf, trang: 30, trong bảng(figure): None, evidence: Nhiễu băng rộng (Wide band noise) ≤20mV rms (5Hz - 1MHz)\n\nNetsure 731 A41 Usermanual.pdf, trang: 30, evidence: Output noise Wide band noise ≤20mV rms (5Hz - 1MHz)<br/>Peak-Peak noise ≤250mV p-p (0 - 20MHz)\n\n']
    }}

In [58]:
product_key_test

{'Bộ chuyển đổi nguồn 220VAC/ 48VDC (kèm theo 02 dàn acquy 200Ah)': {'Yêu cầu chung': ['BCCA8',
   '93C6C',
   '8E2C6',
   '17415',
   'D70B1',
   'ABE44',
   'B4706',
   'Đáp ứng: 4/6 = 67%',
   'Netsure 731 A41 Usermanual.pdf, trang: 11, evidence: The equipment should be unpacked and inspected after it arrives at the installation site. The inspection shall be done by representatives of both the user and Vertiv Co., Ltd. To inspect the equipment, you should open the packing case, take out the packing list and check against the packing list that the equipment is correct and complete. Make sure that the equipment is delivered intact.\n\nnetsure-2100-brochure.pdf, trang: 2, evidence: Specifications are subject to change without notice. November 2018\n\nNetsure-801-brochure.pdf, trang: 4, evidence: | Electrical    | IEC 60950-1, EN 60950-1, UL 60950-1            |\n\nNetsure 731 A41 Usermanual.pdf, trang: 9, trong bảng(figure): None, evidence: The power system is composed of power distrib

In [ ]:
test

In [127]:
!pip install python-docx

In [59]:
from docx import Document
from docx.shared import Pt
from docx.enum.table import WD_TABLE_ALIGNMENT, WD_ALIGN_VERTICAL
# === Tạo Word document ===
doc = Document()
doc.add_heading("BẢNG TUYÊN BỐ ĐÁP ỨNG VỀ KỸ THUẬT", level=1)

# Tạo bảng 6 cột
table = doc.add_table(rows=1, cols=6)
table.style = 'Table Grid'
table.alignment = WD_TABLE_ALIGNMENT.CENTER

# Header
headers = [
    "Hạng mục số", "Tên hàng hoá",
    "Thông số kỹ thuật và các tiêu chuẩn của hàng hoá trong E-HSMT",
    "Thông số kỹ thuật và các tiêu chuẩn của hàng hoá chào trong E-HSDT",
    "Hồ sơ tham chiếu", "Tình đáp ứng của hàng hoá"
]

for i, text in enumerate(headers):
    cell = table.rows[0].cells[i]
    cell.text = text
    for p in cell.paragraphs:
        for run in p.runs:
            run.font.bold = True
            run.font.size = Pt(10)
    cell.vertical_alignment = WD_ALIGN_VERTICAL.CENTER

# Ghi từng dòng
for product, hang_hoa_dict in product_key_test.items():
    for idx, (ten_hang_hoa, items) in enumerate(hang_hoa_dict.items(), start=1):
        ma_ids = items[:-2]  # Các ID
        dap_ung = items[-2]  # ví dụ: "đáp ứng"
        ho_so = items[-1]    # tài liệu tham chiếu

        # Tổng hợp thông số kỹ thuật
        eh_smt = ""
        eh_hsdt = ""

        for ma in ma_ids:
            if ma in test:
                eh_smt += f"- {test[ma]['yeu_cau_ky_thuat_chi_tiet']}\n"
                eh_hsdt += f"- {test[ma].get('kha_nang_dap_ung',"")}\n"

        # Tạo dòng mới
        row = table.add_row().cells
        row[0].text = str(idx)
        row[1].text = ten_hang_hoa
        row[2].text = eh_smt.strip()
        row[3].text = eh_hsdt.strip()
        row[4].text = ho_so
        row[5].text = dap_ung

        for cell in row:
            cell.vertical_alignment = WD_ALIGN_VERTICAL.TOP
            for p in cell.paragraphs:
                for run in p.runs:
                    run.font.size = Pt(9)

# Lưu file
doc.save("bang_tuyen_bo_dap_ung5.docx")